In [1]:
import sys
sys.path.append('../../toyaikit/')

In [14]:
%load_ext autoreload
%autoreload 2

In [2]:
from toyaikit.chat import IPythonChatInterface

In [3]:
from agents import Agent, Runner, SQLiteSession, function_tool

In [4]:
import uuid

In [5]:
runner = Runner()

In [6]:
@function_tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

@function_tool
def dad_joke(name: str) -> str:
    """Tells a dad joke with the provided name."""
    return f"Dad and {name} walk into a bar but there are no empty tables."

agent = Agent(
    name="Bot",
    instructions="You are a helpful assistant. Use tools when needed.",
    tools=[multiply, dad_joke]
)

In [7]:
interface = IPythonChatInterface()

In [12]:
async def run(agent, session):
    while True:
        user_input = interface.input()
        if user_input.lower() == "stop":
            interface.display("Chat ended.")
            break

        result = await runner.run(agent, input=user_input, session=session)
    
        func_calls = {}
    
        for ni in result.new_items:
            raw = ni.raw_item
            if ni.type == 'tool_call_item':
                func_calls[raw.call_id] = raw
            if ni.type == 'tool_call_output_item':
                func_call = func_calls[raw['call_id']]
                interface.display_function_call(func_call.name, func_call.arguments, raw['output'])
            if ni.type == 'message_output_item':
                md = raw.content[0].text
                interface.display_response(md)

In [13]:
session_id = f"chat_session_{uuid.uuid4().hex[:8]}"
session = SQLiteSession(session_id)

await run(agent, session)

You: tell me a joke


You: why do you think I'm alex?


You: I don't care.


You: Give me another one


You: It's the same joke


You: what's 7 times 8


You: sto


You: stop


Chat ended.


## Coding agent

In [8]:
import os
import shutil

@function_tool
def copy_project_template(project_folder: str) -> str:
    """
    copy the template into the project folder

    arguments: project_folder - should be a lowercase string with underscores
    """
    if not project_folder:
        print("Project name cannot be empty.")
        return

    if os.path.exists(project_folder):
        print(f"Directory '{project_folder}' already exists. Please choose a different name or remove the existing directory.")
        return

    shutil.copytree('django_template', project_folder)
    print(f"Django template copied to '{project_folder}' directory.")

    return project_folder

In [9]:
IDEA_PROMPT = """
Ask the user as much as possible about the idea they have. Once you have enough information,
create a copy of the template, and output "Let's build <PROJECT_NAME>", no quotes. 

When coming up with a name for the project, use something creative, to make sure it's unique
and not repeatable.
"""
    
idea_agent = Agent(
    name="IdeaCodingAgent",
    instructions=IDEA_PROMPT,
    tools=[
        copy_project_template
    ]
)

In [10]:
session_id = f"chat_session_{uuid.uuid4().hex[:8]}"
session = SQLiteSession(session_id)

await run(idea_agent, session)

You: I want a todo list


You: just create the best todo list


Django template copied to 'ultimate_todo_masterpiece' directory.


You: stop


Chat ended.


In [11]:
session_items = await session.get_items()

In [12]:
import json

In [14]:
project_folder = None

for item in session_items:
    if item.get('type') == 'function_call':
        if item['name'] == 'copy_project_template':
            project_folder = json.loads(item['arguments'])['project_folder']

if project_folder is None:
    raise "project_folder is not set"

project_folder

'ultimate_todo_masterpiece'

In [15]:
from pathlib import Path
import tools

agent_tools = tools.AgentTools(Path(project_folder))

In [16]:
coding_agent_tools_list = [
    function_tool(agent_tools.execute_bash_command),
    function_tool(agent_tools.read_file),
    function_tool(agent_tools.search_in_files),
    function_tool(agent_tools.see_file_tree),
    function_tool(agent_tools.write_file)
]

In [17]:
DEVELOPER_PROMPT = """
You are a coding agent. Your task is to modify the provided Django project template
according to user instructions. You don't tell the user what to do; you do it yourself using the 
available tools. First, think about the sequence of steps you will do, and then 
execute the sequence.
Always ensure changes are consistent with Django best practices and the project’s structure.

## Project Overview

The project is a Django 5.2.4 web application scaffolded with standard best practices. It uses:
- Python 3.8+
- Django 5.2.4 (as specified in pyproject.toml)
- uv for Python environment and dependency management
- SQLite as the default database (see settings.py)
- Standard Django apps and a custom app called myapp
- HTML templates for rendering views
- TailwindCSS for styling

## File Tree


├── .python-version
├── README.md
├── manage.py
├── pyproject.toml
├── uv.lock
├── myapp/
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations/
│   │   └── __init__.py
│   ├── models.py
│   ├── templates/
│   │   └── home.html
│   ├── tests.py
│   └── views.py
├── myproject/
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── templates/
    └── base.html

## Content Description

- manage.py: Standard Django management script for running commands.
- README.md: Setup and run instructions, including use of uv for dependency management.
- pyproject.toml: Project metadata and dependencies (Django 5.2.4).
- uv.lock: Lock file for reproducible Python environments.
- .python-version: Specifies the Python version for the project.
- myapp/: Custom Django app with models, views, admin, tests, and a template (home.html).
  - migrations/: Contains migration files for database schema.
- myproject/: Django project configuration (settings, URLs, WSGI/ASGI entrypoints).
  - settings.py: Configures installed apps, middleware, database (SQLite), templates, etc.
- templates/: Project-level templates, including base.html.

You have full access to modify, add, or remove files and code within this structure using your available tools.


## Additional instructions

- Don't execute "runserver", but you can execute other commands to check if the project is working.
- Make sure you use Tailwind styles for making the result look beatiful
- Use pictograms and emojis when possible. Font-awesome is awailable
- Avoid putting complex logic to templates - do it on the server side when possible
"""

In [20]:
coding_agent = Agent(
    name="CodingAgent",
    instructions=DEVELOPER_PROMPT,
    tools=coding_agent_tools_list,
    model='gpt-4o-mini'
)

In [21]:
await run(coding_agent, session)

You: go


You: stop


Chat ended.
